## 用Mnist数据集进行分类算法

In [1]:
# 防止内核挂掉
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

In [2]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

导入数据集

In [3]:
# 训练集
train_dataset = datasets.MNIST(root='./data/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

批次设置

In [4]:
# 批次大小
batch_size = 64 # 每次传入数据的量

# 装载训练集
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True) # 打乱数据集

# 装载测试集
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

查看第一个loader中的数据

In [5]:
inputs, labels = next(iter(train_loader))
print('input shape: ', inputs.shape)
print('label shape: ', labels.shape)
print('labels: ', labels)

input shape:  torch.Size([64, 1, 28, 28])
label shape:  torch.Size([64])
labels:  tensor([4, 6, 9, 6, 9, 1, 2, 7, 6, 1, 6, 1, 3, 9, 5, 3, 0, 2, 5, 7, 4, 5, 7, 1,
        9, 7, 8, 4, 2, 3, 5, 4, 6, 7, 6, 8, 2, 8, 6, 8, 4, 5, 1, 6, 2, 8, 4, 0,
        8, 2, 5, 1, 9, 4, 4, 9, 0, 3, 2, 2, 0, 0, 4, 2])


### 网络结构

使用神经网络的框架

In [6]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## Conv2d(通道数, 生成的特征图数, 卷积窗口大小, 步长, padding几圈0)
        ## MaxPool2d(窗口大小, 步长)
        self.conv1 = nn.Sequential(nn.Conv2d(1, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2, 2))
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2, 2))
        self.fc1 = nn.Sequential(nn.Linear(64*7*7, 1000), nn.Dropout(p=0.5), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(1000, 10), nn.Softmax(dim=1))
        
    def forward(self, x):
        # 卷积必须对四维的数据进行操作
        # 本例中数据维度分别为[64, 1, 28, 28](样本数，通道数，长，宽)，可直接传入卷积中
        z1 = self.conv1(x)
        z2 = self.conv2(z1)
        
        # 全连接层则必须是二维数据
        z2 = z2.view(z2.size()[0], -1)
        z3 = self.fc1(z2)
        output = self.fc2(z3)
        
        return output

### 模型参数

In [7]:
# 定义模型
model = Net()

# 损失函数
crossentropy_loss = nn.CrossEntropyLoss()

# 优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 训练与测试的方法

In [8]:
# 训练的方法
def train():
    # 模型的训练状态
    # 模型会Dropout
    model.train()
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 计算loss
        loss = crossentropy_loss(out, labels) # 交叉熵不用one-hot编码
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

In [9]:
# 测试的方法
def test():
    # 模型的验证测试状态
    # 模型不会Dropout
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Test acc: {correct.item()/len(test_dataset)}')

    correct = 0
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Train acc: {correct.item()/len(train_dataset)}')

In [10]:
for epoch in range(20):
    print(f'epoch: {epoch}')
    train()
    test()

epoch: 0
Test acc: 0.9792
Train acc: 0.9777
epoch: 1
Test acc: 0.9821
Train acc: 0.9797166666666667
epoch: 2
Test acc: 0.9868
Train acc: 0.9872
epoch: 3
Test acc: 0.983
Train acc: 0.9861
epoch: 4
Test acc: 0.9895
Train acc: 0.9892666666666666
epoch: 5
Test acc: 0.9871
Train acc: 0.9892
epoch: 6
Test acc: 0.9889
Train acc: 0.9910833333333333
epoch: 7
Test acc: 0.989
Train acc: 0.9912
epoch: 8
Test acc: 0.9908
Train acc: 0.9924
epoch: 9
Test acc: 0.9894
Train acc: 0.9921666666666666
epoch: 10
Test acc: 0.9881
Train acc: 0.9902333333333333
epoch: 11
Test acc: 0.9877
Train acc: 0.98885
epoch: 12
Test acc: 0.9905
Train acc: 0.9908
epoch: 13
Test acc: 0.9866
Train acc: 0.9899166666666667
epoch: 14
Test acc: 0.9828
Train acc: 0.9852166666666666
epoch: 15
Test acc: 0.9919
Train acc: 0.9913166666666666
epoch: 16
Test acc: 0.9897
Train acc: 0.9901833333333333
epoch: 17
Test acc: 0.9874
Train acc: 0.9892333333333333
epoch: 18
Test acc: 0.9884
Train acc: 0.9901
epoch: 19
Test acc: 0.9892
Train acc